In [1]:
import pyupbit
import pandas as pd
import time
import talib

import yfinance as yf
from datetime import date
from database import CRUD

class GetData(object):
    def __init__(self, tickers):
        self._tickers = tickers

    def history_data(self, start:str, end:str):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, start=start, end=end)
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)
        
    def oneday_data(self):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, period='1d')
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)
    
    def onemonth_data(self):
        dfs = []
        for ticker in self._tickers:
            df = yf.download(ticker, period='1mo')
            df['ticker'] = ticker
            dfs.append(df)
        
        df = pd.concat(dfs).sort_index()
        df.index = df.index.strftime("%Y-%m-%d")

        self.add_indicator(df)

        return(df)

    def add_indicator(self, df):
        """
        보조 지표 추가하여 차트 데이터 리턴 

        atr : ATR
        y_atr : 어제기준 ATR
        macd : MACD
        y_macd : 어제기준 MACD
        ma5 : 5일 이동평균선
        ma20 : 20일 이동평균선
        """
        df['y_close'] = df['Close'].shift(1)
        df['atr'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
        df['y_atr'] = df['atr'].shift(1)
        df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['y_macd'], df['y_macdsignal'], df['y_macdhist'] = talib.MACD(df['y_close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['ma5'] = talib.MA(df['Close'], timeperiod=5, matype=0)
        df['ma20'] = talib.MA(df['Close'], timeperiod=20, matype=0)
        
        return df

tickers = ['AAPL', 'NVDA', 'ASML', 'U', 'ABNB']
data = GetData(tickers)
df = data.history_data('2020-03-01', '2021-03-01')
df.tail()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker,y_close,atr,y_atr,macd,macdsignal,macdhist,y_macd,y_macdsignal,y_macdhist,ma5,ma20
Date,,,,,,,,,,,,,,,,,,
2021-02-26,137.524994,138.544998,133.610001,137.145004,137.042099,50093200,NVDA,559.770020,197.106971,179.487506,-1.191540,-7.540873,6.349333,7.505313,-9.128206,16.633519,220.903004,228.526001
2021-02-26,122.589996,124.849998,121.199997,121.260002,120.873283,164560400,AAPL,137.145004,184.166831,197.106971,-9.258915,-7.884481,-1.374434,-1.191540,-7.540873,6.349333,224.245004,224.822002
2021-02-26,572.340027,573.489990,560.700012,567.090027,565.439758,946500,ASML,121.260002,203.314199,184.166831,20.090795,-2.289426,22.380221,-9.258915,-7.884481,-1.374434,313.465010,224.251503
2021-02-26,103.550003,108.000000,101.099998,107.629997,107.629997,7896700,U,567.090027,222.076758,203.314199,6.204545,-0.590632,6.795177,20.090795,-2.289426,22.380221,298.579010,223.333003
2021-02-26,194.179993,214.100006,185.750000,206.350006,206.350006,19368600,ABNB,107.629997,213.819133,222.076758,3.129398,0.153374,2.976024,6.204545,-0.590632,6.795177,227.895007,227.877503


In [127]:
tickers = ['U', 'ABNB']
dfs = []
for ticker in tickers:
    df = yf.download(ticker, period='2mo')
    df['ticker'] = ticker
    df['y_close'] = df['Close'].shift(1)
    df['atr'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['y_atr'] = df['atr'].shift(1)
    df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['y_macd'], df['y_macdsignal'], df['y_macdhist'] = talib.MACD(df['y_close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['ma5'] = talib.MA(df['Close'], timeperiod=5, matype=0)
    df['ma20'] = talib.MA(df['Close'], timeperiod=20, matype=0)
    dfs.append(df)

df = pd.concat(dfs).sort_index()
df.index = df.index.strftime("%Y-%m-%d")
ticker_len = len(tickers)
df.iloc[-ticker_len:,:]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker,y_close,atr,y_atr,macd,macdsignal,macdhist,y_macd,y_macdsignal,y_macdhist,ma5,ma20
Date,,,,,,,,,,,,,,,,,,
2021-10-06,126.610001,130.779999,125.519997,130.380005,130.380005,2541100,U,128.050003,6.041692,6.101822,0.391135,1.833604,-1.442469,0.303099,2.194221,-1.891122,126.150002,131.621001
2021-10-06,161.389999,167.320007,160.630005,167.250000,167.250000,2457800,ABNB,164.740005,5.747868,5.675396,3.217575,4.499567,-1.281992,3.529583,4.820065,-1.290483,167.450000,167.799500


In [123]:
tickers = ['U', 'ABNB']
len(tickers)

2

In [106]:
df

Adj Close         128.05
Adj Close         164.74
Close             128.05
Close             164.74
High              128.18
High              166.59
Low              120.157
Low               162.43
Open              120.81
Open              165.67
Volume           3541600
Volume           4340100
atr               5.6754
atr              6.10182
ma20             131.783
ma20             167.633
ma5               125.24
ma5              167.614
macd            0.303099
macd             3.52958
macdhist        -1.29048
macdhist        -1.89112
macdsignal       2.19422
macdsignal       4.82007
ticker              ABNB
ticker                 U
y_atr            5.93042
y_atr            5.79196
y_close            164.5
y_close           119.85
y_macd          0.423561
y_macd           4.13204
y_macdhist      -1.01064
y_macdhist      -2.24344
y_macdsignal     5.14269
y_macdsignal       2.667
Name: 2021-10-05 00:00:00, dtype: object